In [1]:
import numpy as np
import pandas as pd
import matplotlib, re
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer 
from collections import Counter
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from nltk.stem.porter import PorterStemmer 

## Import and clean the data 

In [62]:
tweets = pd.read_csv("vaccination_all_tweets.csv")
tweets = tweets.dropna(axis=0) # TODO: discuss
tweets.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
6,1337851215875608579,Gunther Fehlinger,"Austria, Ukraine and Kosovo",End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is a bit sad to claim the fame for success ...,['vaccination'],Twitter Web App,0,4,False
9,1337842295857623042,Ch.Amjad Ali,Islamabad,#ProudPakistani #LovePakArmy #PMIK @insafiansp...,2012-11-12 04:18:12,671,2368,20469,False,2020-12-12 19:30:33,#CovidVaccine \n\nStates will start getting #C...,"['CovidVaccine', 'COVID19Vaccine', 'US', 'paku...",Twitter Web App,0,0,False
10,1337841934170255365,Tamer Yazar,Turkey-Israel,"Im Market Analyst, also Editor... working (fre...",2009-09-17 16:45:16,1302,78,339,False,2020-12-12 19:29:07,"while deaths are closing in on the 300,000 mar...","['PfizerBioNTech', 'Vaccine']",Twitter Web App,0,0,False


In [3]:
# number of missing values
tweets.isnull().sum()

id                  0
user_name           0
user_location       0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
retweets            0
favorites           0
is_retweet          0
dtype: int64

In [5]:
#X_train, X_test, y_train, y_test = train_test_split(processed_features, test_size = 0.2, random_state=123)

In [6]:
# remove emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

# source : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b


In [7]:
# source : https://realpython.com/sentiment-analysis-python/
def tokenize(text):
    return text.split()


In [75]:
tweets = pd.read_csv("vaccination_all_tweets.csv")
tweets = tweets.dropna(axis=0) # TODO: discuss
original_text = tweets["text"].copy(deep=True)
original_text.iloc[6]

'UPDATED: #YellowFever &amp; #COVID19 #ImmunityPassports - Part Two\n\n#SARSCoV2 #PfizerBioNtech #Britain #December… https://t.co/qKT7Rst9aW'

In [76]:
# clean text d
def clean_text(text):
#     text = str.lower(text) #lower case
    text = re.sub(r"http\S+", "", text) #rm websites
    text = re.sub(r"\s+[a-zA-Z]\s+", " ", text) # rm single characters 
    text = re.sub(r"\s\s+","", text) # rm multiple spaces
    text = re.sub(r"#","", text) # rm hashtags
    text = re.sub(r"[(,?!)]","", text) # rm special characters 
    text = re.sub("@[^\s]+", "", text) # rm @twitter_id
#     text = remove_emoji(text) # rm emoji
    #text = re.sub("<[^>]*>", "", text) # rm HTML markup
    emoticons = re.findall("(?::|;|=)(?:-)?(?:\)|\(|D|P)", text)
    text = (re.sub("[\W]+", " ", text.lower()) + " ".join(emoticons).replace("-", "")).strip()
    # lower case since we assume letter case does not contain any significant information
    #text = text.split()
    #corpus = [txt for txt in text if not txt in set(stopwords.words('english'))]
    #text = tokenize(text)
                
    #TODO : remove stop words or not? : expensive
    return text

tweets["text"] = tweets["text"].apply(lambda x: clean_text(x))
tweets["text"]

0        same folks said daikon paste could treat cytok...
2        coronavirus sputnikv astrazeneca pfizerbiontec...
6        it is bit sad to claim the fame for success of...
9        covidvaccinestates will start getting covid19v...
10       while deaths are closing in on the 300000 mark...
                               ...                        
33708    zakharova russia has offered foreign embassy s...
33712    breakingthe information attack targeting sputn...
33713    pity as my personal preference would have been...
33714    who in close contact with russia on sputnikv c...
33716    breakingus targets europe in smear campaign ag...
Name: text, Length: 19616, dtype: object

## Vectorize
### CountVectorizer constructs a bag-of-words model baesd on word counts

In [77]:
count = CountVectorizer()
docs = np.array(tweets["text"])
bag = count.fit_transform(docs) # constructed bow model and transformed sentences into sparse feature vectors

In [78]:
count.vocabulary_ # each vocab is stored in a dict that maps unique words to integer indices

{'same': 16293,
 'folks': 7806,
 'said': 16259,
 'daikon': 5471,
 'paste': 13853,
 'could': 4740,
 'treat': 19087,
 'cytokine': 5453,
 'storm': 17839,
 'pfizerbiontech': 14064,
 'coronavirus': 4653,
 'sputnikv': 17549,
 'astrazeneca': 2131,
 'moderna': 12119,
 'covid_19': 5062,
 'russian': 16181,
 'vaccine': 19789,
 'is': 10086,
 'created': 5288,
 'to': 18850,
 'last': 10821,
 'years': 21046,
 'it': 10123,
 'bit': 2891,
 'sad': 16223,
 'claim': 4071,
 'the': 18575,
 'fame': 7422,
 'for': 7835,
 'success': 17968,
 'of': 13249,
 'vaccination': 19745,
 'on': 13337,
 'patriotic': 13884,
 'competition': 4368,
 'between': 2716,
 'usa': 19606,
 'canada': 3531,
 'uk': 19340,
 'and': 1683,
 'covidvaccinestates': 5194,
 'will': 20760,
 'start': 17682,
 'getting': 8263,
 'covid19vaccine': 5020,
 'monday': 12274,
 'us': 19604,
 'sayspakustv': 16412,
 'nyc': 13164,
 'healthcare': 8908,
 'globalgoals': 8352,
 'while': 20699,
 'deaths': 5617,
 'are': 1949,
 'closing': 4152,
 'in': 9606,
 '300000': 54

In [79]:
bag.toarray() # each idx position corresponds to the integer values shown above 
# represents raw term frequencies, the # of times a term occurs in a document
# 1 gram by default

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### Term frequency inverse document frequency (td-idf) 
Assesses word relevancy. 
Downweight frequently occuring words in the feature vectors. 
The product of the term frequency and the inverse document frequency

In [19]:
np.array(tweets["text"])

array(['same folks said daikon paste could treat a cytokine storm pfizerbiontech https t co xehhimg1kf',
       ' coronavirus sputnikv astrazeneca pfizerbiontech moderna covid_19 russian vaccine is created to last 2 4 years https t co ieylckbr8p',
       'it is a bit sad to claim the fame for success of vaccination on patriotic competition between usa canada uk and https t co ifmraygytp',
       ...,
       'pity as my personal preference would have been astrazeneca s vaccine but i was offered pfizer instead wait un https t co dyofx1o0xn',
       ' who in close contact with russia on sputnikv certification https t co vnb9x2hgsr',
       ' breaking us targets europe in smear campaign against russia s sputnikv vaccine kremlin source says https t co 6qk1mfj4fs'],
      dtype=object)

In [81]:
tfidf = TfidfTransformer(use_idf=True, norm = "l2", smooth_idf=True)
np.set_printoptions(precision=2)
print(tfidf.fit_transform(count.fit_transform(docs).toarray()))
# takes raw term frequencies and transforms them into tf-idfs

  (0, 19087)	0.34538737638702904
  (0, 17839)	0.361508442680639
  (0, 16293)	0.2488614621387622
  (0, 16259)	0.20731649915602854
  (0, 14064)	0.11905241428192843
  (0, 13853)	0.4051886538318002
  (0, 7806)	0.2949520744015946
  (0, 5471)	0.4051886538318002
  (0, 5453)	0.4051886538318002
  (0, 4740)	0.2267229781489089
  (1, 21046)	0.37929960802236873
  (1, 19789)	0.12191963009225507
  (1, 18850)	0.13833376764115266
  (1, 17549)	0.2016554268180789
  (1, 16181)	0.3166725902604512
  (1, 14064)	0.18088023427496505
  (1, 12119)	0.15140922325781944
  (1, 10821)	0.3418119336822565
  (1, 10086)	0.1770877490060499
  (1, 5288)	0.47962076404297654
  (1, 5062)	0.3347691319454065
  (1, 4653)	0.2551370697277343
  (1, 2131)	0.2648499292931064
  (2, 19745)	0.15307835969043934
  (2, 19606)	0.24468703032064504
  :	:
  (19613, 2586)	0.18531251278604063
  (19613, 2131)	0.1732455403514765
  (19613, 2035)	0.16425029740517186
  (19614, 20802)	0.2146247833276023
  (19614, 20712)	0.2656365707013829
  (19614, 175

In [86]:
## tokenizer splits the text corpora into individual elements
## word stemming: transforming a word into its root form (maps related words  to the same stem)

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

tweets["stemmed"] = tweets["text"].apply(lambda x: tokenizer_porter(x))

#### Remove stop words

In [89]:
nltk.download("stopwords")

def remove_stop_words(text):
    stop = stopwords.words("english")
    return [word for word in text if word not in stop]

tweets["stop_removed"] = tweets["stemmed"].apply(lambda x: remove_stop_words(x))

[nltk_data] Downloading package stopwords to /Users/shiny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [88]:
tweets["stemmed"].head()

0     [same, folk, said, daikon, past, could, treat,...
2     [coronaviru, sputnikv, astrazeneca, pfizerbion...
6     [it, is, bit, sad, to, claim, the, fame, for, ...
9     [covidvaccinest, will, start, get, covid19vacc...
10    [while, death, are, close, in, on, the, 300000...
Name: stemmed, dtype: object

In [90]:
tweets["stop_removed"].head()

0     [folk, said, daikon, past, could, treat, cytok...
2     [coronaviru, sputnikv, astrazeneca, pfizerbion...
6     [bit, sad, claim, fame, success, vaccin, patri...
9     [covidvaccinest, start, get, covid19vaccin, mo...
10    [death, close, 300000, mark, million, peopl, w...
Name: stop_removed, dtype: object

In [94]:
tweets[:3]

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,stemmed,stop_removed
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20 06:06:44,same folks said daikon paste could treat cytok...,['PfizerBioNTech'],Twitter for Android,0,0,False,"[same, folk, said, daikon, past, could, treat,...","[folk, said, daikon, past, could, treat, cytok..."
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12 20:33:45,coronavirus sputnikv astrazeneca pfizerbiontec...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False,"[coronaviru, sputnikv, astrazeneca, pfizerbion...","[coronaviru, sputnikv, astrazeneca, pfizerbion..."
6,1337851215875608579,Gunther Fehlinger,"Austria, Ukraine and Kosovo",End North Stream 2 now - the pipeline of corru...,2013-06-10 17:49:22,2731,5001,69344,False,2020-12-12 20:06:00,it is bit sad to claim the fame for success of...,['vaccination'],Twitter Web App,0,4,False,"[it, is, bit, sad, to, claim, the, fame, for, ...","[bit, sad, claim, fame, success, vaccin, patri..."


## Document classification

In [95]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
param_grid = [{"vect_ngram_range": [(1,1)],
              "vect_stop_words":[stop,None],
              "vect_tokenizer":[tokenizer,
                               tokenizer_porter],
              "clf_penalty":[11,12],
              "clf_C":[1.0, 10.0, 100.0]},
              {"vect_ngram_range":[(1,1)],
               "vect_stop_words": [stop, None],
               "vect_tokenizer": [tokenizer,
                                 tokenizer_porter],
               "vect_use_idf": [False],
               "vect_norm":[None],
               "clf_penalty": ["l1", "l2"],
               "clf_C": [1.0, 10.0, 100.0]}]
lr_tfidf = Pipeline([("vect", tfidf),
                     ("clf", LogisticRegression(random_state = 0, solver= "liblinear"))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,scoring = "accuracy",
                          cv = 5, verbose=2, n_jobs = -1)
gs_lr_tfidf.fit(X_train, y_train)

## Sentiment analysis - Christine's part

In [ ]:
def polarity(text):
    testimonial = TextBlob(text)
    polarity = testimonial.sentiment.polarity
    return polarity


def subjectivity(text):
    testimonial = TextBlob(text)
    subjectivity = testimonial.subjectivity
    return subjectivity


def senti(text, polarity_threshold=0):
    testimonial = TextBlob(text)
    senti = testimonial.sentiment.polarity
    
    if senti >= polarity_threshold:
        return 1 # positive
    elif np.abs(senti) == polarity_threshold:
        return 0
    else:
        return 0 #negative
    

tweets['polarity'] = tweets['text'].apply(lambda x: polarity(x))
tweets['subjectivity'] = tweets['text'].apply(lambda x: subjectivity(x))
tweets['sentiment'] = tweets['text'].apply(lambda x: senti(x))

In [ ]:
tweets.head()

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(tweets["text"], tweets["sentiment"], 
#                                                     test_size = 0.2, random_state=123)

In [ ]:
vectorizer = TfidfVectorizer(max_features = 3000, min_df= 10, stop_words = stopwords.words("english"))
processed_features = vectorizer.fit_transform(tweets["text"]).toarray()
processed_features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(processed_features, tweets["sentiment"], 
                                                    test_size = 0.2, random_state=123)

In [ ]:
text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)
predictions = text_classifier.predict(X_test)
print(accuracy_score(y_test,predictions))

In [ ]:
print(tweets.info())

In [ ]:
# EDA: plot locations 
most_location = tweets["user_location"].value_counts().head(15)
fig, ax = plt.subplots()
most_location.plot.bar()
fig.suptitle("Number and Percentages of Tweets Based on Location")
ax.set_xlabel("Locations")
ax.set_ylabel("Number of tweets")
type(ax)

In [ ]:
# clean hashtags
tweets["hashtags"] = tweets["hashtags"].apply(lambda x: re.sub("'","",x))
tweets["hashtags"] = tweets["hashtags"].apply(lambda x: re.sub("\[","",x))
tweets["hashtags"] = tweets["hashtags"].apply(lambda x: re.sub("\]","",x))
tweets["hashtags"] = tweets["hashtags"].apply(lambda x: x.split(","))

In [ ]:
# most common hashtags 
hashtag_dict = {}
for hashtag_list in tweets["hashtags"]:
    for hashtag in hashtag_list:
        if hashtag.strip().lower() not in hashtag_dict:
            hashtag_dict[hashtag.strip().lower()] = 0
        hashtag_dict[hashtag.strip().lower()] += 1
most_common_hashtag = sorted(hashtag_dict.items(), key = lambda x : x[1], reverse=True)
most_common_hashtag[:20]

In [ ]:
# Mean of 2.37 hashtags
tweets["hashtags_count"] = tweets["hashtags"].apply(lambda x: len(x))
np.mean(tweets["hashtags_count"]) 

In [ ]:
followers_and_hashtags = tweets[["user_followers","hashtags_count"]]

In [ ]:
fig, ax = plt.subplots()
followers_and_hashtags.plot(x="user_followers", y = "hashtags_count",
                            kind="scatter", ax = ax)
ax.set_xlabel("Number of User Followers (in 1e7)")
ax.set_ylabel("Number of Hashtags")
ax.set_title("Follower numbers vs Number of Hashtags")

In [ ]:
# most common words
words_dict = {}
for text in tweets["text"].values:
    for word in text.split():
        if word in stopwords.words("english"):
            continue
        if word not in words_dict:
            words_dict[word] = 0
        words_dict[word] += 1 
most_common = sorted(words_dict.items(), key = lambda x : x[1], reverse=True)
most_common[:10]

In [ ]:
# word cloud text

all_tweets = "".join(rev for rev in tweets["text"])
fig, ax = plt.subplots(figsize = (10,10))
wordcloud_img = WordCloud(max_font_size = 40, max_words = 50, background_color = "white").generate(all_tweets)
ax.imshow(wordcloud_img)
ax.axis("off")